# Import packages
run the following cell to import necessary packages to run the analysis

In [1]:
import os
from pathlib import Path

from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
# from pylibCZIrw import czi as pyczi
# from bioio import BioImage
# import bioio_czi

# Reading data

In [2]:
data_path = Path('/mnt/Data/nuno_martins/Image_data/Ex011_Re01/ExM/sample_data/')
filelist = os.listdir(data_path)
print(len(filelist))

2


In [3]:
filelist = [x for x in filelist if x.find('.tif')>0]
filelist.sort()
filelist

['NPM_Ex011_Re01_a2_postExM_Im-01_AcquisitionBlock4-AiryscanProcessing_sample1.tif',
 'NPM_Ex011_Re01_a2_postExM_Im-01_AcquisitionBlock4-AiryscanProcessing_sample2.tif']

In [ ]:
file_index = 0
file_path = str(data_path)+'/'+filelist[file_index]
filename = filelist[file_index][:filelist[file_index].index('.czi')]
print(file_path)
print(filename)

In [5]:
save_path = Path(str(data_path)+'/split_czi')
save_path.mkdir(parents=True, exist_ok=True)
print(save_path)

/mnt/Data/nuno_martins/Image_data/Ex010_Re01/ExM/20250325/test_dir/split_czi


___
# Read raw data
reading raw tile data from `czi` tile without stitching for visualization.

In [4]:
from tifffile import imread

In [5]:
image_data = imread(str(data_path) + '/' + filelist[0])

In [6]:
image_data.shape

(353, 2, 1451, 1451)

In [17]:
from bioio import BioImage
# import bioio_czi

In [7]:
from bioio.writers.ome_zarr_writer_2 import OmeZarrWriter, compute_level_shapes, compute_level_chunk_sizes_zslice

In [11]:
image_shape = (1, image_data.shape[1], image_data.shape[0], image_data.shape[2], image_data.shape[3])
image_shape

(1, 2, 353, 1451, 1451)

In [14]:
scaling = (1, 1, 2, 2, 2)
num_levels = 2

In [16]:
shapes = compute_level_shapes(image_shape, scaling, num_levels)
shapes

[(1, 2, 353, 1451, 1451), (1, 2, 176, 725, 725)]

In [18]:
chunks = compute_level_chunk_sizes_zslice(shapes)
chunks

[(1, 1, 1, 1451, 1451), (1, 1, 4, 725, 725)]

In [19]:
writer = OmeZarrWriter()
writer.init_store(str(data_path), shapes, chunks, image_data.dtype)

In [22]:
image_data_save = np.moveaxis(image_data, 0, 1)
image_data_save = np.expand_dims(image_data_save, 0)
image_data_save.shape

(1, 2, 353, 1451, 1451)

In [24]:
writer.write_t_batches_array(image_data_save, tbatch=4)

In [35]:
meta = writer.generate_metadata(
    image_name=filelist[0][:filelist[0].index('.tif')],
    channel_names=['Cy5', 'DAPI'],
    physical_dims={"t": 1, "z":0.8, "y":0.047, "x":0.047},
    physical_units={"t":"minute", "z":"micrometer", "y":"micrometer", "x":"micrometer"},
    # channel_colors=['FF0000', '0080ff'],
    # channel_colors=['red', 'cyan'],
    channel_colors=[255, 125],
)

In [36]:
writer.write_metadata(meta)